<a target="_blank" href="https://colab.research.google.com/github/jackma-00/house-price-prediction/blob/main/experiments/4_house_price_online_inference.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# <span style="font-width:bold; font-size: 3rem; color:#333;">Online Inference Pipeline</span>

Installing packages

In [1]:
!pip uninstall scikit-learn -y

Found existing installation: scikit-learn 1.6.0
Uninstalling scikit-learn-1.6.0:
  Successfully uninstalled scikit-learn-1.6.0


In [2]:
!pip install scikit-learn==1.5.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 45.0 MB/s eta 0:00:00


In [3]:
!pip install xgboost==2.1.3

In [4]:
!pip install hopsworks[python]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 8.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 74.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 80.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.6/258.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 640.3/640.3 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### <span style='color:#ff5f27'> Imports

In [15]:
from xgboost import XGBRegressor
from datetime import datetime
import pandas as pd
import numpy as np
import hopsworks
import uuid
import os

import warnings
warnings.filterwarnings("ignore")

Hopsworks API Key

In [6]:
# If you haven't set the env variable 'HOPSWORKS_API_KEY', then uncomment the next line and enter your API key
os.environ["HOPSWORKS_API_KEY"] = "DMT7cBmSbXxvrmlm.SGi5E7zfqXqjsMJWWgiJFpiMlQep8mMiP5hAlvVCIVBXw5nCOzV67kVhGxIua122"

## <a class="anchor" id="1.5_bullet" style="color:#ff5f27"> 🚀 Model Deployment</a>

### <span style="color:#ff5f27;">📎 Predictor script for Python models</span>

In [ ]:
%%writefile predict_house_price.py

class Predict(object):

    def __init__(self):
        """ Initializes the serving state, reads a trained model"""
        # Get feature store handle
        project = hopsworks.login()
        self.mr = project.get_model_registry()
        self.xgboost_regressor = XGBRegressor()

        # Retrieve the feature view from the model
        retrieved_model = self.mr.get_model(
            name="house_price_xgboost_model",
            version=15,  # Latest version
        )

        # Download the saved model artifacts to a local directory
        saved_model_dir = retrieved_model.download()

        self.feature_view = retrieved_model.get_feature_view()  # house_price_fv v4 attached to the model

        # Load the trained model
        self.xgboost_regressor.load_model(saved_model_dir + "/model.json")
        print("Initialization Complete")

    def predict(self, inputs):
        """ Serves a prediction request usign a trained model"""
        feature_vector = self.feature_view.get_feature_vector(inputs)
        feature_vector = feature_vector.drop(columns=["price"])

        return self.model.predict(feature_vector).tolist() # Numpy Arrays are not JSON serializable

In [7]:
proj = hopsworks.login()
fs = proj.get_feature_store()
mr = proj.get_model_registry()


Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1158295


### <span style='color:#ff5f27'> Download the model from the model registry

In [9]:
retrieved_model = mr.get_model(
    name="house_price_xgboost_model",
    version=15,                        # Latest version
)

# Download the saved model artifacts to a local directory
saved_model_dir = retrieved_model.download()

In [10]:
# Loading the XGBoost regressor model from the saved model directory
# retrieved_xgboost_model = joblib.load(saved_model_dir + "/xgboost_regressor.pkl")
retrieved_xgboost_model = XGBRegressor()

retrieved_xgboost_model.load_model(saved_model_dir + "/model.json")

# Displaying the retrieved XGBoost regressor model
retrieved_xgboost_model

XGBRegressor(base_score='3.3713372E5', booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None,
             feature_types=['float', 'float', 'float', 'float', 'float', 'int',
                            'float', 'float', 'float', 'float', 'int', 'int',
                            'int', 'int', 'int', 'int', 'in...
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [11]:
# Get feature view
feature_view = retrieved_model.get_feature_view()  # house_price_fv v4 attached to the model

### <span style='color:#ff5f27'> Get Inference data

In [98]:
# Mock inference data
inference_data = {
    #"id": 187693421,
    #"scraping_date": datetime.today().date(),
    "agencyid": 169110.0,
    "bedroomsnumber": 3.0,
    "buildingyear": 2023.0,
    "codcom": 26086.0,
    "gsm": 181.0,
    "surface": 253,
    "latitude": 45.6674,
    "longitude": 12.244,
    "isluxury": [True],
    "isnew": [False],
    "on_the_market": [False],
    "zeroenergybuilding": [False],
    "airconditioning": "autonomo, freddo",
    "bathrooms": "3",
    "city": "Treviso",
    "condition": "Nuovo / In costruzione",
    "energyclass": "A2",
    "ga4heating": "Autonomo",
    "garage": "1 in box privato/box in garage",
    "heatingtype": "autonomo, a pavimento",
    "pricerange": "oltre 500.000 &euro;",
    'id_zona_omi':"F704-B11",
    "rooms": "4",
    #"price": 0.0,
}


In [32]:
# Intialize predicted price to match feature view schema
inference_data["price"] = 0.0

In [33]:
# Incorporate uuid
def generate_numeric_uuid():
    # Generate a UUID and extract its integer form
    raw_uuid = uuid.uuid4()
    numeric_uuid = str(raw_uuid.int)[:9]  # Take the first 9 digits
    return numeric_uuid

# Generate the numeric UUID
genrated_uuid = generate_numeric_uuid()

primary_key = {"id": int(genrated_uuid)}

# Add serving key as first key in the dictionary
inference_data.update(primary_key)

In [34]:
# Generate the current event timestamp
current_timestamp = datetime.today()

inference_data["scraping_date"] = current_timestamp.date()

In [99]:
inference_data

{'agencyid': 169110.0,
 'bedroomsnumber': 3.0,
 'buildingyear': 2023.0,
 'codcom': 26086.0,
 'gsm': 181.0,
 'surface': 253,
 'latitude': 45.6674,
 'longitude': 12.244,
 'isluxury': [True],
 'isnew': [False],
 'on_the_market': [False],
 'zeroenergybuilding': [False],
 'airconditioning': 'autonomo, freddo',
 'bathrooms': '3',
 'city': 'Treviso',
 'condition': 'Nuovo / In costruzione',
 'energyclass': 'A2',
 'ga4heating': 'Autonomo',
 'garage': '1 in box privato/box in garage',
 'heatingtype': 'autonomo, a pavimento',
 'pricerange': 'oltre 500.000 &euro;',
 'id_zona_omi': 'F704-B11',
 'rooms': '4'}

In [48]:
feature_view.primary_keys

{'id', 'id_zona_omi'}

In [101]:
# Apply Model-dependent transformations to the inference data
transformed_data = feature_view.get_feature_vector(
    entry={'id_zona_omi': inference_data['id_zona_omi']},
    passed_features=inference_data,
    return_type="list",
)

In [102]:
transformed_data

[169110.0,
 3.0,
 2023.0,
 26086.0,
 181.0,
 253,
 45.6674,
 12.244,
 1915.0,
 2663.0,
 0    1
 dtype: int64,
 0    0
 dtype: int64,
 0    0
 dtype: int64,
 0    0
 dtype: int64,
 4,
 3,
 87,
 2,
 0,
 3,
 0,
 0,
 1,
 541,
 5,
 4,
 0,
 0]

In [ ]:
transformed_data

### <span style='color:#ff5f27'> Making the predictions

In [103]:
predicted_price = retrieved_xgboost_model.predict(transformed_data)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (28,) + inhomogeneous part.

### <span style='color:#ff5f27'> Saving the predictions (for monitoring) to a feature group

In [ ]:
# Update price with actual predicted price
inference_data["price"] = predicted_price

In [ ]:
# Incorporate uuid
def generate_numeric_uuid():
    # Generate a UUID and extract its integer form
    raw_uuid = uuid.uuid4()
    numeric_uuid = str(raw_uuid.int)[:9]  # Take the first 9 digits
    return numeric_uuid

# Generate the numeric UUID
genrated_uuid = generate_numeric_uuid()

inference_data["id"] = genrated_uuid

In [ ]:
# Generate the current event timestamp
current_timestamp = datetime.now()

inference_data["timestamp"] = current_timestamp

In [ ]:
# Get or create the 'property_preds' feature group
property_preds = fs.get_or_create_feature_group(
    name='property_preds',
    description='Properties predicted prices',
    version=2,
    primary_key=['id'],
    online_enabled=True,
    event_time='timestamp'
)

In [ ]:
# Insert data into feature group
property_preds.insert(inference_data)